# This file: Transforms cleaned CA school enrollment data into (aggregated and lagged) format needed for analysis
# Dependencies: enrollment, predicted_flows80, predicted_flows90
# Outputs: agg_enrollment_CDS, agg_enrollment_CD, enrollment_IV_SD, enrollment_IV_SD_90base
# Last updated: 2/21/2019

In [1]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/Users/tsengtammy/anaconda/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Read files
sd = pd.read_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/enrollment.csv')

/Users/tsengtammy/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,6,21,22,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
# Read files
predicted_flows80 = pd.read_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/predicted_flows80.csv')
predicted_flows90 = pd.read_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/predicted_flows90.csv')

In [22]:
# Lite cleaning
for df in [predicted_flows80, predicted_flows90]:
    df['District'] = df['NAME'].apply(lambda x: x[:-16])
    df.rename(columns={'Year':'YEAR'}, inplace=True)

# Def transforms

In [63]:
def transform(df, id_column):

    # pivot, group
    grouped = df.copy().groupby([id_column, 'ETHNIC_CAT', 'YEAR'])
    race_by_dist = grouped.sum()['ENR_TOTAL'].reset_index()
    race_by_dist = race_by_dist.pivot_table(
        index=[id_column,'YEAR'], columns='ETHNIC_CAT', values='ENR_TOTAL').reset_index()

    # rename
    race_by_dist.rename(columns={'White, not Hispanic':'White'}, inplace=True)

    # shares of white
    race_by_dist['Total'] = race_by_dist.drop([id_column, 'YEAR'], axis=1).sum(axis=1)
    race_by_dist['L'] = race_by_dist['Asian'] + race_by_dist['White']
    race_by_dist['White_share'] = race_by_dist['White'] / race_by_dist['Total']
    race_by_dist['Asian_share'] = race_by_dist['Asian'] / race_by_dist['Total']
    race_by_dist['Other_share'] = 1 - race_by_dist['White_share'] - race_by_dist['Asian_share']
    race_by_dist = race_by_dist.sort_values([id_column, 'YEAR'], ascending=True)
    
    # get lags and append 
    # row is t - (t-1)
    lags = race_by_dist.diff()
    lags[id_column] = race_by_dist[id_column]
    lags['YEAR'] = race_by_dist['YEAR'] # keep as t - (t-1)
    lags.rename(columns=lambda x: x + '_t-(t-1)', inplace=True)
    lags.rename(columns={'{}_t-(t-1)'.format(id_column): id_column,
                        'YEAR_t-(t-1)':'YEAR'}, inplace=True)

    # merge the lags data
    master = race_by_dist.merge(lags, on=[id_column, 'YEAR'], how='left')
    

    # merge back county names / pops
    county_mapping = df[[id_column, 'YEAR', 'County', 'County_Population', 
                         'District', 'DOC', 'DOCType']].drop_duplicates(subset=[id_column, 'YEAR']).reset_index().drop('index', axis=1)
    master = master.merge(county_mapping, on=[id_column, 'YEAR'], how='left')
    print(len(master))
    
    # get the (t-1) versions of the levels
    diff1 = master[[id_column, 'YEAR', 'American Indian or Alaska', 'Asian', 'Black, not Hispanic', 'Filipino',
                   'Hispanic or Latino', 'Pacific Islander', 'White', 'Total', 'L', 'White_share', 'Asian_share',
                   'Other_share', 'County_Population']].copy()
    diff1['YEAR'] = diff1['YEAR'] + 1 # to get (t-1)
    diff1.rename(columns=lambda x: x + '_(t-1)', inplace=True)
    diff1.rename(columns={'{}_(t-1)'.format(id_column): id_column,
                        'YEAR_(t-1)':'YEAR'}, inplace=True)

    master = master.merge(diff1, on=[id_column, 'YEAR'], how='left')
    print(len(master))
    
    return master

# Transform at school level

In [64]:
schools = transform(sd, 'CDS_CODE')
schools.head()

307864
307864


,CDS_CODE,YEAR,American Indian or Alaska,Asian,"Black, not Hispanic",Filipino,Hispanic or Latino,Pacific Islander,White,Total,...,Filipino_(t-1),Hispanic or Latino_(t-1),Pacific Islander_(t-1),White_(t-1),Total_(t-1),L_(t-1),White_share_(t-1),Asian_share_(t-1),Other_share_(t-1),County_Population_(t-1)
0,1100170109835,2005,3.0,225.0,112.0,27.0,67.0,8.0,222.0,664.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1100170109835,2006,9.0,352.0,145.0,31.0,112.0,29.0,326.0,1004.0,...,27.0,67.0,8.0,222.0,664.0,447.0,0.334337,0.338855,0.326807,1462736.0
2,1100170109835,2007,17.0,385.0,117.0,21.0,99.0,29.0,392.0,1060.0,...,31.0,112.0,29.0,326.0,1004.0,678.0,0.324701,0.350598,0.324701,1462371.0
3,1100170109835,2008,5.0,429.0,125.0,9.0,75.0,19.0,397.0,1059.0,...,21.0,99.0,29.0,392.0,1060.0,777.0,0.369811,0.363208,0.266981,1470622.0
4,1100170109835,2009,6.0,282.0,146.0,25.0,179.0,29.0,731.0,1398.0,...,9.0,75.0,19.0,397.0,1059.0,826.0,0.374882,0.405099,0.220019,1484085.0


In [65]:
# SAVE FOR ANALYSIS
schools.to_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/agg_enrollment_CDS.csv', index=False)

# Transform at district level (all districts)

In [66]:
districts = transform(sd, 'CD Code')
districts.head()

36658
36658


,CD Code,YEAR,American Indian or Alaska,Asian,"Black, not Hispanic",Filipino,Hispanic or Latino,Pacific Islander,White,Total,...,Filipino_(t-1),Hispanic or Latino_(t-1),Pacific Islander_(t-1),White_(t-1),Total_(t-1),L_(t-1),White_share_(t-1),Asian_share_(t-1),Other_share_(t-1),County_Population_(t-1)
0,110017,1981,2.0,20.0,220.0,11.0,NaN,NaN,135.0,388.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110017,1984,5.0,13.0,225.0,4.0,NaN,NaN,65.0,312.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110017,1985,4.0,15.0,290.0,4.0,NaN,2.0,69.0,384.0,...,4.0,NaN,NaN,65.0,312.0,78.0,0.208333,0.041667,0.750000,1170400.0
3,110017,1986,6.0,13.0,346.0,2.0,NaN,1.0,71.0,439.0,...,4.0,NaN,2.0,69.0,384.0,84.0,0.179688,0.039062,0.781250,1185500.0
4,110017,1987,7.0,14.0,373.0,4.0,NaN,5.0,58.0,461.0,...,2.0,NaN,1.0,71.0,439.0,84.0,0.161731,0.029613,0.808656,1206900.0


In [67]:
# SAVE FOR ANALYSIS
districts.to_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/agg_enrollment_CD.csv', index=False)

# Subset to just elem + unsd districts + add instrument (80, 90 base)

In [68]:
# Only keep ELEM_UNSD
elem_unsd = districts[(districts['DOC']==52) | (districts['DOC']==54)].copy()

In [76]:
# 80 base for 80 data
# can also use as 80 base for 90 data by just subsetting
inst80 = elem_unsd.merge(predicted_flows80, on=['District', 'YEAR'], how='left', indicator=True)
inst80 = inst80[inst80['_merge']=='both']
inst80.to_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/enrollment_IV_SD_80base.csv', index=False)

In [77]:
# 90 base for 90 data
inst90 = elem_unsd.merge(predicted_flows90, on=['District', 'YEAR'], how='left', indicator=True)
inst90 = inst90[inst90['_merge']=='both']
inst90.to_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/enrollment_IV_SD_90base.csv', index=False)

# Transform at district level (top 100 districts out of all)

In [78]:
grouped = districts.groupby('CD Code').mean().reset_index()[['CD Code', 'Total']]
top_dist = grouped.sort_values('Total', ascending=False)[:100]['CD Code']

districts[districts['CD Code'].isin(top_dist)].to_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/top_100_cd.csv', index=False)

# Transform at district level (top 100 districts out of elem + unsd) - w/ 80 base data

In [79]:
grouped = inst80.groupby('CD Code').mean().reset_index()[['CD Code', 'Total']]
top_dist = grouped.sort_values('Total', ascending=False)[:100]['CD Code']

inst80[inst80['CD Code'].isin(top_dist)].to_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/top_100_cd_iv80.csv', index=False)

# Transform at district level (top 100 districts out of elem + unsd) - w/ 90 base data

In [80]:
grouped = inst90.groupby('CD Code').mean().reset_index()[['CD Code', 'Total']]
top_dist = grouped.sort_values('Total', ascending=False)[:100]['CD Code']

inst90[inst90['CD Code'].isin(top_dist)].to_csv('/Users/tsengtammy/Dropbox/3 TT/4. Thesis/build/data/output/top_100_cd_iv90.csv', index=False)